<a href="https://colab.research.google.com/github/shivigup/IUAC-Phase-control/blob/main/code/BC_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import glob, os
import matplotlib.pyplot as plt
import scipy
import scipy.stats

In [3]:
os.chdir("drive/MyDrive/Summer 2022/IUAC/data/Usable")
dictionary = []
for a in ["19F_15112021.csv", "19F_15112021_2.csv", "28Si_23112021_1.csv", "28Si_23112021_2.csv"]:
  dictionary.append(pd.read_csv(a))

In [4]:
phase_column_header = 'Phase Meter-Phase'
for i in range(len(dictionary)):
  dictionary[i] = dictionary[i].drop(['DATE/TIME'], axis=1)
  if(dictionary[i][phase_column_header].dtype!='float64'):
    dictionary[i] = dictionary[i].drop([0], axis=0)
    df_length = dictionary[i].shape[0]
    for j in range(1, df_length+1):
      if((dictionary[i][phase_column_header][j]).endswith('\n')):
        dictionary[i][phase_column_header][j] = dictionary[i][phase_column_header][j][:-2]
  dictionary[i][phase_column_header] = pd.to_numeric(dictionary[i][phase_column_header])

In [5]:
for i in range(len(dictionary)):
  to_drop = []
  for j in dictionary[i].columns:
    if(dictionary[i][j].isnull().values.any()):
      to_drop.append(j)
      print(j)
  dictionary[i] = dictionary[i].drop(columns = to_drop)

GS_T_1
GS_T_1


In [6]:
locked_val = []
for df in dictionary:
  mean = scipy.stats.circmean(df[phase_column_header], high=360, low=0)
  locked_val.append(mean)

In [ ]:
locked_val

In [8]:
for i in range(len(dictionary)):
  dictionary[i]['phase_difference'] = (dictionary[i][phase_column_header] - locked_val[i])%360 - 180
  dictionary[i] = dictionary[i].drop([phase_column_header], axis=1)

In [9]:
def Intersection(lst1, lst2):
    return set(lst1).intersection(lst2)

final_columns = dictionary[0].columns
for i in range(1, len(dictionary)):
  final_columns = Intersection(dictionary[i].columns, final_columns)

In [10]:
final_columns = list(final_columns)
final_columns

['INJ_VOLTAGE',
 'IMLSN1',
 'Energy',
 'GVM',
 'SF6_pressure',
 'TYPE_0',
 'EQ_021(X)',
 'ESP021',
 'ES_011(Y-VRN)',
 'ES_021(X-VRN)',
 'ES_022(X)',
 'ES_011(X-VRN)',
 'CHARGE_STATE_0',
 'ES_021(X)',
 'CATS21',
 'TYPE_1',
 'ES_021(Y)',
 'TOTAL_UP_CHARGE',
 'ES_021(Y-VRN)',
 'Phase Meter-Freq',
 'phase_difference',
 'ES_022(Y-VRN)',
 'CHP021',
 'ES_022(Y)',
 'HVSI11',
 'ES_022(X-VRN)',
 'ES_011(Y)',
 'EQ_021(Y)',
 'CHARGE_STATE_1',
 'ES_011(X)']

In [11]:
minmax = np.empty([len(final_columns), 2])

In [12]:
for i in range(len(final_columns)):
  minmax[i, 0] = min(dictionary[0][final_columns[i]])
  minmax[i, 1] = max(dictionary[0][final_columns[i]])

In [13]:
for i in range(1, len(dictionary)):
  for j in range(len(final_columns)):
    mi = min(dictionary[i][final_columns[j]])
    ma = max(dictionary[i][final_columns[j]])
    if(mi<minmax[j, 0]):
      minmax[j, 0] = mi
    if(ma>minmax[j, 1]):
      minmax[j, 1] = ma

In [14]:
to_remove = minmax[:,0]==minmax[:,1]
final_columns = [d for (d, remove) in zip(final_columns, to_remove) if not remove]
minmax = [d for (d, remove) in zip(minmax, to_remove) if not remove]

In [15]:
for df in dictionary:
  for i in range(len(final_columns)):
    df[final_columns[i]] = 10*(df[final_columns[i]] - minmax[i][0])/(minmax[i][1] - minmax[i][0])

In [16]:
# testing for window_size = 5, 10, 15, 20

window_size = 20;
data_dim = len(final_columns)

In [17]:
X = np.empty([0, window_size, data_dim])
y = pd.DataFrame(columns = final_columns)

In [18]:
X

array([], shape=(0, 20, 29), dtype=float64)

In [19]:
for df in dictionary:
  for i in range(window_size, len(df)):
    if(np.abs(df.loc[i, 'phase_difference']) < np.abs(df.loc[i-1, 'phase_difference'])):
      x_i = df.loc[i-window_size:i-1, final_columns].to_numpy()
      x_i = x_i.reshape(1, x_i.shape[0], x_i.shape[1])
      X = np.append(X, x_i, axis=0)
      y = y.append(df.loc[i, final_columns])

In [20]:
y = y.drop(['phase_difference'], axis=1)

In [21]:
print(X.shape)
print(y.shape)

(20977, 20, 29)
(20977, 28)


In [22]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [23]:
y = np.array(y)

In [24]:
scores = np.array([])

In [25]:
import tensorflow

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

In [ ]:
for i in range(5):
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=10*i)
  model = Sequential()
  model.add(LSTM(data_dim-1, activation='relu', kernel_initializer='he_normal', input_shape=(window_size, data_dim)))
  #model.add(LSTM(30, activation='relu', kernel_initializer='he_normal'))
  model.add(Dense(data_dim-1))
  #model.add(Dense(data_dim-1))
  model.compile(optimizer='adam', loss='mae')
  history = model.fit(X_train, y_train, epochs=300, batch_size=16, verbose=1)
  plt.plot(history.history['loss']) 
  score = model.evaluate(X_test, y_test)
  scores = np.append(scores, score)

Epoch 1/300
879/879 [==============================] - 11s 11ms/step - loss: 1.6685
Epoch 2/300
879/879 [==============================] - 9s 10ms/step - loss: 0.1919
Epoch 3/300
879/879 [==============================] - 9s 10ms/step - loss: 0.1444
Epoch 4/300
879/879 [==============================] - 10s 11ms/step - loss: 0.1216
Epoch 5/300
879/879 [==============================] - 10s 11ms/step - loss: 0.1027
Epoch 6/300
879/879 [==============================] - 10s 11ms/step - loss: 0.0889
Epoch 7/300
879/879 [==============================] - 9s 10ms/step - loss: 0.0834
Epoch 8/300
879/879 [==============================] - 9s 10ms/step - loss: 0.0793
Epoch 9/300
879/879 [==============================] - 9s 11ms/step - loss: 0.0760
Epoch 10/300
879/879 [==============================] - 9s 10ms/step - loss: 0.0744
Epoch 11/300
879/879 [==============================] - 11s 12ms/step - loss: 0.0728
Epoch 12/300
879/879 [==============================] - 9s 11ms/step - loss: 0.0

In [ ]:
scores.mean()

In [ ]:
scores.std()

In [ ]:
scores